# Person re-identification 

### This sample use simple pretrained resnet-152 model to extract features/embeddings for an image 

### Compute Embedding

In [1]:
images_dir = "./tests/imagesMarket1501/"

In [2]:
import sys
import os
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader

sys.path.append(os.path.abspath('./src'))
from resnetembedding import ResnetEmbedder
from datasets.Market1501Dataset import Market1501Dataset
from EuclideanPairwiseDistance import EuclideanPairwiseDistance
from CMCScore import CMCScore


In [3]:
%%time 

dataset = Market1501Dataset(images_dir)
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

model = ResnetEmbedder()

embeddings = []
class_person = []
    
for person_img, target in dataloader:
    embedding = model(person_img)
    embeddings.extend(embedding)
    class_person.extend(target)

embeddings = torch.stack(embeddings)
class_person = torch.stack(class_person)


tensor([0, 0])
tensor([1, 1])
tensor([1, 1])
CPU times: user 710 ms, sys: 204 ms, total: 914 ms
Wall time: 974 ms


In [ ]:
class_person

In [ ]:
embeddings[0:5]

In [ ]:
embeddings.shape

### Compute pairwise distance

In [ ]:
distance_metric = EuclideanPairwiseDistance()
pairwise_distance = distance_metric(embeddings)
pairwise_distance

### Compute CMC score
Definition: https://www.nist.gov/sites/default/files/documents/2016/12/06/12_ross_cmc-roc_ibpc2016.pdf

* Each sample is  compared against all gallery samples. The resulting scores are sorted and ranked

* Determine the rank at which a true match occurs 
* True Positive Identification Rate (TPIR): Probability of observing the correct identity within the top K ranks
* CMC Curve: Plots TPIR against ranks
* CMC Curve: Rank based metric



In [ ]:
from EuclideanPairwiseDistance import EuclideanPairwiseDistance

distance_metric = EuclideanPairwiseDistance()
pairwise_distance = distance_metric(embeddings)
pairwise_distance


In [ ]:
class_person

In [ ]:
from CMCScore import CMCScore

cmc_scorer = CMCScore()
score = cmc_scorer.score(pairwise_distance, class_person, 1)
print(score)

In [ ]:
k=1
accuracy, rank_k, rank_k_label = cmc_scorer.accuracy_at_top_k(pairwise_distance,  class_person, k)
print("Percentage accuracy at k = {} is {}".format(k, accuracy ))

In [ ]:
rank_k

In [ ]:
result_df = pd.DataFrame(rank_k_label, columns= ["predicted_k_{}".format(i) for i in range(1, k+1)])
result_df["actual"] = class_person

In [ ]:
result_df.head(n=20)

### Visualise the embeddings extracted using T-SNE

In [ ]:
!pip install scikit-learn==0.21.2

In [ ]:
from sklearn.manifold import TSNE
embedding_tsne = TSNE(random_state=42).fit_transform(embeddings)

print("embeddings reduced to 2-D using TSNE\n")
print(embedding_tsne[0:10])

In [ ]:
group = list(set(class_person))
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

scatter = ax.scatter(embedding_tsne[:, 0], embedding_tsne[:, 1], c= class_person, alpha=0.5, s= 200, cmap="Spectral" )

plt.title('T-SNE plot of person embedding')
plt.show()